In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from pandas import Series,DataFrame

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
%matplotlib inline

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import train_test_split

In [4]:
random_seed = 1111

In [5]:
train_x_raw = pd.read_csv("G:/Redhat_share/ML_micro/train_x.csv")
train_y_raw = pd.read_csv("G:/Redhat_share/ML_micro/train_y.csv")
train_xy_raw = pd.merge(train_x_raw,train_y_raw,on='uid')
test_x = pd.read_csv("G:/Redhat_share/ML_micro/test_x.csv")
features_type = pd.read_csv("G:/Redhat_share/ML_micro/features_type.csv")
test_uid = test_x.uid
train_xy_raw.head()

,uid,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x1130,x1131,x1132,x1133,x1134,x1135,x1136,x1137,x1138,y
0,1792,5.04,5.77,6.04,3.91,0.66,1.04,1.83,5.41,5.45,...,1,20,4,1,7,15,6,6,6,1
1,4211,5.98,2.90,1.94,1.42,0.10,3.29,0.81,9.09,8.04,...,8,1,2,7,7,3,8,7,24,1
2,14658,7.04,8.92,3.24,0.01,0.00,4.50,0.00,8.41,6.97,...,6,1,1,6,8,6,7,4,19,1
3,17041,13.30,7.16,5.69,1.98,0.71,2.20,0.22,13.89,1.75,...,1,20,1,1,8,1,6,8,9,1
4,15765,5.24,5.09,5.31,1.05,0.00,4.24,0.34,5.25,5.59,...,1,3,1,1,8,31,1,6,14,1


In [6]:
train_xy = train_xy_raw.drop(['uid'],axis=1)
train,val = train_test_split(train_xy_raw, test_size = 0.2,random_state=random_seed)
train_y = train.y
train_x = train.drop(['y'],axis=1)
val_y = val.y
val_x = val.drop(['y'],axis=1)

In [8]:
dtrain = xgb.DMatrix(train_x, label=train_y)
dval = xgb.DMatrix(val_x,label=val_y)
dtest = xgb.DMatrix(test_x)
params={
    'booster':'gbtree',
    'early_stopping_rounds':100,
    'scale_pos_weight': 5,
    'objective': 'binary:logistic',
    'colsample_bytree':0.4,
    
    'gamma':0.05,
    'lambda':600,
    'max_depth':7,
    'subsample':0.6,
    'min_child_weight':3,
    'eta': 0.02,
    'seed':random_seed,
    'nthread':16,
    'eval_metric': 'auc'
    }

In [9]:
watchlist  = [(dval,'val'), (dtrain,'train')]
model = xgb.train(params,dtrain,num_boost_round=5000,evals=watchlist)
model.save_model('xgb.model')
#预测
test_y = model.predict(dtest,ntree_limit=model.best_ntree_limit)
test_result = pd.DataFrame(columns=["uid","score"])
test_result.uid = test_uid
test_result.score = test_y
test_result.to_csv("result.csv",index=None,encoding='utf-8')

[0]	val-auc:0.499439	train-auc:0.503283
[1]	val-auc:0.501008	train-auc:0.509759
[2]	val-auc:0.501794	train-auc:0.514276
[3]	val-auc:0.502206	train-auc:0.518569
[4]	val-auc:0.502802	train-auc:0.521075
[5]	val-auc:0.501523	train-auc:0.522402
[6]	val-auc:0.500786	train-auc:0.524910
[7]	val-auc:0.500784	train-auc:0.524915
[8]	val-auc:0.502520	train-auc:0.528697
[9]	val-auc:0.503662	train-auc:0.530152
[10]	val-auc:0.503113	train-auc:0.531230
[11]	val-auc:0.503702	train-auc:0.533796
[12]	val-auc:0.506763	train-auc:0.537956
[13]	val-auc:0.506003	train-auc:0.539526
[14]	val-auc:0.509641	train-auc:0.542614
[15]	val-auc:0.511413	train-auc:0.545386
[16]	val-auc:0.510899	train-auc:0.546915
[17]	val-auc:0.510347	train-auc:0.547837
[18]	val-auc:0.511633	train-auc:0.549150
[19]	val-auc:0.510923	train-auc:0.552141
[20]	val-auc:0.511547	train-auc:0.552420
[21]	val-auc:0.511548	train-auc:0.552420
[22]	val-auc:0.511548	train-auc:0.552421
[23]	val-auc:0.511251	train-auc:0.552624
[24]	val-auc:0.511095	trai